In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

# LangSmith Tracing and Tracking
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")
os.environ["LANGSMITH_TRACING"] = os.getenv("LANGSMITH_TRACING")

In [3]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field
from typing import Optional

llm = ChatGroq(model="gemma2-9b-it")

class Product(BaseModel):
    name : str = Field(..., description = "Name of the Product")
    details : str = Field(..., description = "Core Details related to the Product")
    price : int = Field(..., description = "Tentaive Price for the Product in $ (Dollars)")

parser = PydanticOutputParser(pydantic_object = Product)
format_instructions = parser.get_format_instructions()

chat_prompt = ChatPromptTemplate.from_messages([
    ('system', "You are an Product Assistant. Your task is to extract relevant information about the product given following instructions: \n{format_instructions} "),
    ('human', "Product : {product}")
])

##### Without Chaining (for understanding flow)

In [4]:
final_prompt = chat_prompt.invoke({
    'format_instructions' : format_instructions,
    'product' : "Nvidia RTX 4080"
})
print(final_prompt.messages)

[SystemMessage(content='You are an Product Assistant. Your task is to extract relevant information about the product given following instructions: \nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "Name of the Product", "title": "Name", "type": "string"}, "details": {"description": "Core Details related to the Product", "title": "Details", "type": "string"}, "price": {"description": "Tentaive Price for the Product in $ (Dollars)", "title": "Price", "type": "integer"}}, "required": ["name", "details", "price"]}\n``` ', additional_kwargs={}, resp

In [5]:
result = llm.invoke(final_prompt)
print(result.content)

```json
{
  "name": "Nvidia RTX 4080",
  "details": "High-performance graphics card",
  "price": 1199
}
```


In [6]:
final_result = parser.parse(result.content)
print(final_result)

name='Nvidia RTX 4080' details='High-performance graphics card' price=1199


##### With Chaining

In [7]:
chain = chat_prompt | llm | parser

chain_result = chain.invoke({
    'format_instructions' : format_instructions,
    'product' : "Nvidia RTX 4080"
})
print(chain_result)

name='Nvidia RTX 4080' details='High-performance graphics card' price=1200


Overall Workflow

In [8]:
graph = chain.get_graph()
print(graph.draw_ascii())

    +-------------+      
    | PromptInput |      
    +-------------+      
            *            
            *            
            *            
 +--------------------+  
 | ChatPromptTemplate |  
 +--------------------+  
            *            
            *            
            *            
      +----------+       
      | ChatGroq |       
      +----------+       
            *            
            *            
            *            
+----------------------+ 
| PydanticOutputParser | 
+----------------------+ 
            *            
            *            
            *            
      +---------+        
      | Product |        
      +---------+        
